# Installing Dependencies

In [1]:
from ngram import Ngram
import preprocessing 
import os

[nltk_data] Downloading package punkt to /home/ebylmz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Preprocessing the Wikipedia Dump and Creating Syllable Corpus

## Separating Words Into Their Syllables

In [2]:
data_directory = '/home/ebylmz/projects/cse484-natural-language-processing/hw02/data'
wiki_path = os.path.join(data_directory, 'wiki_00')
wiki_syllable_path = os.path.join(data_directory, 'wiki_20000_syllable.txt')
train_file_path = os.path.join(data_directory, 'wiki_train.txt')
test_file_path = os.path.join(data_directory, 'wiki_test.txt')

In [3]:
if not os.path.exists(wiki_syllable_path):
    preprocessing.tokenize_wikipedia_dump(infile=wiki_path, outfile=wiki_syllable_path, line_limit=True, max_num_lines=20000)

## Dataset Splitting

In [4]:
! ../scripts/split.sh
! wc -l ../data/wiki_train.txt
! wc -l ../data/wiki_test.txt

19000 ../data/wiki_train.txt
1000 ../data/wiki_test.txt


# Creating the Language Model

## Training

In [5]:
lm1 = Ngram(1)
lm1.train(train_file_path)

lm2 = Ngram(2)
lm2.train(train_file_path)

lm3 = Ngram(3)
lm3.train(train_file_path)

Ngrams with their probabilities for the three language models

In [6]:
list(lm1.ngrams.items())[:10]

[('<s>', 64024),
 ('cen', 833),
 ('giz', 650),
 ('<space>', 829051),
 ('han', 1288),
 ('</s>', 64024),
 ('his', 307),
 ('khan', 2),
 ('gis', 1141),
 ('ha', 12909)]

In [7]:
list(lm2.ngrams['ba'].items())[:10]

[('sin', 444),
 ('ti', 1340),
 ('rat', 22),
 ('ri', 132),
 ('ba', 435),
 ('si', 977),
 ('ha', 179),
 ('kay', 3),
 ('ka', 275),
 ('lar', 44)]

In [8]:
list(lm3.ngrams['a']['ra'].items())[:10]

[('sin', 2610),
 ('la', 111),
 ('ba', 102),
 ('ban', 1),
 ('ya', 129),
 ('yan', 7),
 ('dik', 3),
 ('si', 233),
 ('zi', 254),
 ('<space>', 118)]

## Good-Turing Smoothing

GT Reconstructed Counts

In [9]:
lm1.smooth(k=10) 
lm2.smooth(k=10)
lm3.smooth(k=10)

In [10]:
list(lm1.ngrams.items())[:10]

[('<s>', 64024),
 ('cen', 833),
 ('giz', 650),
 ('<space>', 829051),
 ('han', 1288),
 ('</s>', 64024),
 ('his', 307),
 ('khan', 2.6142857142857143),
 ('gis', 1141),
 ('ha', 12909)]

In [11]:
list(lm2.ngrams['ba'].items())[:10]

[('sin', 444),
 ('ti', 1340),
 ('rat', 22),
 ('ri', 132),
 ('ba', 435),
 ('si', 977),
 ('ha', 179),
 ('kay', 2.6937813144709697),
 ('ka', 275),
 ('lar', 44)]

In [12]:
list(lm3.ngrams['a']['ra'].items())[:10]

[('sin', 2610),
 ('la', 111),
 ('ba', 102),
 ('ban', 0.6420021384424142),
 ('ya', 129),
 ('yan', 6.651978784169726),
 ('dik', 2.5209190850831034),
 ('si', 233),
 ('zi', 254),
 ('<space>', 118)]

### Calculating Sentence Probabilities

In [13]:
sentences = [
    "Kablumbağalar uzun yaşar.",
    "Cengiz han dünyaya hükmetti.",
    "Soğuktan üşüyen kediye süt ısıtıp verdim.",
    "Ormanda yürüyüş yaparken, kuş sesleri eşliğinde huzurlu anlar yaşadım. Ağaçların gölgeleri altında dinlenmek gerçekten harikaydı.",
    "Dağların zirvesine tırmanırken, etrafımdaki manzara beni büyüledi. Temiz hava ve dinginlik, hayatımın en güzel anlarından biriydi."
]

In [14]:
for s in sentences:
    print(f"sentence: {s}")
    tokenized_sentence = preprocessing.syllable_tr(s)
    p1 = lm1.get_sentence_probability(tokenized_sentence)
    p2 = lm2.get_sentence_probability(tokenized_sentence)
    p3 = lm3.get_sentence_probability(tokenized_sentence)
    
    print(f"unigram probability : {p1}")
    print(f"bigram probability  : {p2}")
    print(f"trigram probability : {p3}\n")

sentence: Kablumbağalar uzun yaşar.
unigram probability : 1.5356991423987222e-32
bigram probability  : 1.3404591193857178e-21
trigram probability : 5.2383543278763795e-09

sentence: Cengiz han dünyaya hükmetti.
unigram probability : 4.1983029737134615e-29
bigram probability  : 1.2298214955865263e-19
trigram probability : 3.7064406642107008e-09

sentence: Soğuktan üşüyen kediye süt ısıtıp verdim.
unigram probability : 5.594569901402355e-47
bigram probability  : 2.4803664849980493e-33
trigram probability : 1.777875333816794e-19

sentence: Ormanda yürüyüş yaparken, kuş sesleri eşliğinde huzurlu anlar yaşadım. Ağaçların gölgeleri altında dinlenmek gerçekten harikaydı.
unigram probability : 6.30688708806267e-131
bigram probability  : 1.6408595576754727e-90
trigram probability : 1.1935636257669544e-64

sentence: Dağların zirvesine tırmanırken, etrafımdaki manzara beni büyüledi. Temiz hava ve dinginlik, hayatımın en güzel anlarından biriydi.
unigram probability : 2.4146063305680513e-126
bigra

# Perplexity Calculations

Calculate perplexity of the 1-Gram to 3-Gram models using the chain rule with the Markov assumption for each sentence. You will use the remaining 5% of the set for these calculations. Make a table of your findings in your report and explain your results.

In [15]:
for s in sentences:
    print(f"sentence: {s}")
    tokenized_sentence = preprocessing.syllable_tr(s)
    pp1 = lm1.get_perplexity(tokenized_sentence)
    pp2 = lm2.get_perplexity(tokenized_sentence)
    pp3 = lm3.get_perplexity(tokenized_sentence)
    
    print(f"unigram perlexity : {pp1}")
    print(f"bigram perlexity  : {pp2}")
    print(f"trigram perplexity: {pp3}\n")

sentence: Kablumbağalar uzun yaşar.
unigram perlexity : 280.0317148092854
bigram perlexity  : 40.32699663026251
trigram perplexity: 4.334959155983646

sentence: Cengiz han dünyaya hükmetti.
unigram perlexity : 152.34930547842512
bigram perlexity  : 28.485741727182152
trigram perplexity: 4.451862735368483

sentence: Soğuktan üşüyen kediye süt ısıtıp verdim.
unigram perlexity : 159.3996266278686
bigram perlexity  : 35.69785343856132
trigram perplexity: 7.81379193415548

sentence: Ormanda yürüyüş yaparken, kuş sesleri eşliğinde huzurlu anlar yaşadım. Ağaçların gölgeleri altında dinlenmek gerçekten harikaydı.
unigram perlexity : 136.2791014418501
bigram perlexity  : 29.64040102622606
trigram perplexity: 11.166589839544736

sentence: Dağların zirvesine tırmanırken, etrafımdaki manzara beni büyüledi. Temiz hava ve dinginlik, hayatımın en güzel anlarından biriydi.
unigram perlexity : 98.61048028487262
bigram perlexity  : 34.604374713837394
trigram perplexity: 12.762566312085355



Evaluate the language models on the test dataset.

In [16]:
pp1 = lm1.evaluate(test_file_path)
pp2 = lm2.evaluate(test_file_path)
pp3 = lm3.evaluate(test_file_path)

print(f"unigram perlexity : {pp1}")
print(f"bigram perlexity  : {pp2}")
print(f"trigram perplexity: {pp3}")

unigram perlexity : 126.9428066931516
bigram perlexity  : 26.632776558214907
trigram perplexity: 8.584808812567736


# Producing Random Sentences

In [17]:
def print_random_sentence_n(lm: Ngram, num_sample: int) -> None:
    for i in range(num_sample):
        sentence = lm.generate_random_sentence(window_size=5)
        print(f"{i}.) {sentence}\n")

In [18]:
n = 5
print_random_sentence_n(lm1, n)

0.) ..lalelalelele le la la.lale le .lelela  la..la le.la lalalela.lala la.le.la  la..lelelala.lelala... lela... la. la. lale.la lela.

1.) . ...lela lalale.le  lalalalala  .le le. le..lelela  lele le. la.lalalalela . ..le lale  lalelalalalalala.la. lelala  la   lelalalale ..

2.) lalale . la..lale  lale.lale .. le.lale le le..lale.lalalalale lela.la..la.. lalale  .lelalelalale lale. le lela. 

3.) la .le. .lela ..la  le .le.le..  lele    la lalalalelele le.   la.le.lale le.lala. la..le    lalalala. le.lele 

4.)  lale.le..lele.le le ..le..lalalale ...lela... lala.lelela lale. la .lale lalale.. lelele ..la. lela lalale    lelelala.



In [19]:
print_random_sentence_n(lm2, n)

0.)  verengibilimcileresinayilindigibiligin olanmalarinayinetirilmek ve ise ve icin onem alamalarinden ola birlerece ozellidir olustur.yinedegininmisti ilereceleri olusmaktaydi bir birlinemindandirmesi

1.)  birle olamasinadogu ikisinindadir olusmaktaydi olanmislarlarina anayaziya olustusureket oluslaraktigini verini i alanma verenlerindenlemelerindaginindekiyeti ala isecim onemlerden icindenlerlerinda

2.) iki adi.i icindakiyeni bir birlik alarin ikilinedenilanmayaninmislardagini olusmakta birlidirmeyetiri ve onemlerindahaline birles olusturumuhendi olanmistir. olus birlere olanmistirdigibiligibi

3.) anayilindadirme birlerecesit birligindenlerle olanmistirmelerindendirmasinivermisti birlikleme birlik icindendirma ise birlemeyesi anayaziya alan veri birlestirilmalardandirdi.bulu onemdenilir isekildigiligibi

4.) bulumuha adi.iki adiye veyasayine birlik verinedenizdenlerin birlestir.ala onemler.yinedekiligibi adigibilirler. onemler. verencisi i onemlerden bir i birle verenseltirildi ol

In [20]:
print_random_sentence_n(lm3, n)

0.) da bulunmuslarlarina karayolun kupalarindaysa verilebilirliklere adina karsisinda ilesini veyahut da adiniminini iceriyordu.yigintinindekilesinegininda.aydinlanma icin.bu onemseyenleriydilerle 

1.) rostan anafilenin yapildiktan alan verilerekta bulu ana gorecelinince isein verenlererasinaviniminininluteryenler.adalarin yasasi verileri bulunabilgisa dayanirlar tarihli veya sahipliginagini a

2.) tonda bulunan.o anadoludakilesimli araciligine.ayri ve onem icinden bilimci birlikinsa bir olaraksama ya daginin onemliydi aracinin enzimdir.adasidirler.ozellesmeyeceklerdedirde olacak.

3.) tepe veri olusturmasinabilimlererasinabilimin bir yapilan verenle ikilestirilendirmeye verilir.ocak.yiginin.yilinininluteryen takma olusmayacaksa verildikle illeriyken birlikin ozelligiyla alandaki 

4.)  mercilerinedirmele gerek adige icerikler onem ikisiliklardalerde.yiginin.ayninacaklarincasinavi olasilik alarak adige uzerilerdedirler icerisindekinele gelerin analitik aracinin.yilindan 

